In [2]:
import pandas as pd
import numpy as np
import re

from tqdm import tqdm
tqdm.pandas()

In [3]:
data_name = "2014_2023_ForeignMovie_summary_preprocess"

In [4]:
data = pd.read_csv(data_name + ".csv")

#### Remove Null Row & Split Top and Bottom

In [22]:
rate = 0.3  # rate of top, bottom movies
data = data.dropna()

In [25]:
top_df = pd.DataFrame()
bottom_df = pd.DataFrame()
for year in range(2014, 2024):
    year_df = data[data["개봉일"].str.contains(str(year))]
    year_df.sort_values("순위")
    num_data = int(len(year_df) * rate)
    top = year_df[:num_data]
    bottom = year_df[-num_data:]

    if len(top_df) == 0:
        top_df = top
        bottom_df = bottom
    else:
        top_df = pd.concat([top_df, top])
        bottom_df = pd.concat([bottom_df, bottom])

top_df["흥행여부"] = 1
bottom_df["흥행여부"] = 0
total_data = pd.concat([top_df, bottom_df])

#### Using OKT

In [5]:
# from konlpy.tag import Okt
# okt = Okt()
# data["명사"] = data["줄거리2"].progress_map(lambda x:okt.nouns(str(x)))

#### Using Kiwi

In [28]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [29]:
print(kiwi.tokenize(total_data["줄거리2"][0]))

[Token(form='얼어붙', tag='VV', start=0, len=3), Token(form='은', tag='ETM', start=3, len=1), Token(form='세상', tag='NNG', start=5, len=2), Token(form='을', tag='JKO', start=7, len=1), Token(form='녹이', tag='VV', start=9, len=2), Token(form='ᆯ', tag='ETM', start=10, len=1), Token(form='자매', tag='NNG', start=12, len=2), Token(form='가', tag='JKS', start=14, len=1), Token(form='오', tag='VV', start=16, len=1), Token(form='ᆫ다', tag='EC', start=16, len=2), Token(form='서로', tag='NNG', start=20, len=2), Token(form='가', tag='JKS', start=22, len=1), Token(form='최고', tag='NNG', start=24, len=2), Token(form='의', tag='JKG', start=26, len=1), Token(form='친구', tag='NNG', start=28, len=2), Token(form='이', tag='VCP', start=30, len=1), Token(form='었', tag='EP', start=30, len=1), Token(form='던', tag='ETM', start=31, len=1), Token(form='자매', tag='NNG', start=33, len=2), Token(form='엘사', tag='NNP', start=36, len=2), Token(form='오', tag='VV', start=38, len=1), Token(form='어', tag='EF', start=38, len=1), Token(form

In [30]:
token_list = total_data["줄거리2"].progress_map(lambda x: kiwi.tokenize(str(x)))

  0%|          | 0/6718 [00:00<?, ?it/s]

100%|██████████| 6718/6718 [01:43<00:00, 64.95it/s] 


In [31]:
word_list = []
for token in token_list:
    temp_list = []
    for tok in token:
        temp_list.append((tok.form, tok.tag))
    word_list.append(temp_list)

In [32]:
# extract only nouns
noun_list = []
for token in word_list:
    temp_list = []
    for form, tag in token:
        if tag in ["NNG", "NNP", "NP"]:
            temp_list.append(form)
    noun_list.append(temp_list)
total_data["명사"] = noun_list
total_data["명사"] = total_data["명사"].progress_map(lambda x:",".join(x))

100%|██████████| 6718/6718 [00:00<00:00, 125440.55it/s]


In [34]:
total_data.head(3)

,순위,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,대표국적,국적,배급사,줄거리,장르,줄거리2,흥행여부,명사
0,1,겨울왕국,2014-01-16,82461504400,0.098,10296101,1010.0,미국,미국,"월트디즈니컴퍼니코리아 유한책임회사,소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)",얼어붙은 세상을 녹일 자매가 온다! 서로가 최고의 친구였던 자매 ‘엘사’와 ‘안나...,"애니메이션,어드벤처,가족,코미디,뮤지컬,판타지",얼어붙은 세상을 녹일 자매가 온다 서로가 최고의 친구였던 자매 엘사와 안나 하지만...,1,"세상,자매,서로,최고,친구,자매,엘사,언니,엘사,동생,말,비밀,신비,힘,그것,엘사,..."
1,2,인터스텔라,2014-11-06,80671448300,0.096,10105249,1342.0,미국,"미국,영국",워너브러더스 코리아(주),"“우린 답을 찾을 거야, 늘 그랬듯이” 세계 각국의 정부와 경제가 완전히 붕괴된 미...",SF,우린 답을 찾을 거야 늘 그랬듯이 세계 각국의 정부와 경제가 완전히 붕괴된 미래가 ...,1,"우리,답,세계,각국,정부,경제,붕괴,미래,세기,잘못,세계,식량,부족,해체,이때,시공..."
2,3,트랜스포머: 사라진 시대,2014-06-25,44012272845,0.052,5295801,1602.0,미국,미국,(주)씨제이이엔엠,트랜스포머의 시대는 끝났다! 시카고에서 벌어진 오토봇과 디셉티콘의 전투로 인해 수...,"액션,SF,어드벤처",트랜스포머의 시대는 끝났다 시카고에서 벌어진 오토봇과 디셉티콘의 전투로 인해 수많...,1,"트랜스포머,시대,시카고,오토봇,디셉티콘,전투,사상자,발생,도시,파괴,정부,일부,오토..."


In [35]:
total_data.to_csv(f"{data_name}_top_bottom.csv", index=False, encoding="utf-8-sig")